In [1]:
import six_layers as model
import torch
import numpy as np
from torch.utils.data import DataLoader
import os, sys

In [2]:
currentdir = os.path.dirname(os.path.abspath(''))
parentdir = os.path.dirname(currentdir)
grandparentdir = os.path.dirname(parentdir)
sys.path.append(grandparentdir)

from res.process_data.dataset import tensor_dataset
from res.fnn.training import training_bce
import res.process_data.process_output as pro_out
import res.fnn.functions as func

from res.process_data.dataset import tensor_dataset


/Users/dengnaicheng/Developer/intro_GANs/gansForPolymer/Naicheng/res/fnn


In [3]:
input_coorinate = np.load('../../data/coordinates.npy')
input_three_dirc = np.load('../../data/three_directions.npy')

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
z_dim = 8
lr = 0.0001
gen = model.GeneratorNet()
gen_opt = torch.optim.Adam(gen.parameters(), lr = lr)
disc = model.DiscriminatorNet()
disc_opt = torch.optim.Adam(disc.parameters(), lr = lr)

In [6]:
check_point_gen = torch.load('6_layers/model_gan_2000.pth.tar')
# check_point_disc = torch.load('6_layers/model_disc_2000.pth.tar')
gen.load_state_dict(check_point_gen['gen_state_dict'])
# disc.load_state_dict(check_point_disc['disc_state_dict'])

<All keys matched successfully>

In [16]:
shuffle = True
batch_size = 125
num_worker = 0
pin_memory = True
input_tensor = torch.Tensor(input_three_dirc)
my_dataset = tensor_dataset(input_tensor, 15, 1)
my_dataloader = DataLoader(dataset= my_dataset,
                            shuffle=shuffle,
                            batch_size=batch_size,
                            num_workers=num_worker,
                            pin_memory=pin_memory)

In [17]:
from Naicheng.res.fnn.training import training_bce, training_wloss
import Naicheng.res.process_data.process_output as pro_out
display_step = 782

In [18]:
import json
# 定义一个workflow
# 每训练100次，保存模型，保存输出
# save all the loss value
loss_value_disc = {}
loss_value_gen = {}
n_critic = 5
clip_value = 0.01
total_epoch = 2000
epoch_step = 100
for i in range(10):
    pro_out.save_model(gen, disc, 'model', total_epoch)
    # disc_loss, gen_loss = training_bce(gen, disc, z_dim, epoch_step, my_dataloader, device, disc_opt, gen_opt,
    #              display_step)
    training_wloss(gen, disc, my_dataloader, epoch_step, z_dim,
                   batch_size, device, disc_opt, gen_opt, n_critic, clip_value)
    total_epoch += epoch_step
    # loss_value_disc[f'epoch{total_epoch}'] = disc_loss
    # loss_value_gen[f'epoch{total_epoch}'] = gen_loss
with open('gen_loss.json', 'w') as fp:
    json.dump(loss_value_gen, fp)
with open('disc_loss.json', 'w') as fp:
    json.dump(loss_value_disc, fp)